### Project 1

We need to design an dimplement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1050.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

There are going to be many ways to design something like this, especially since I have not nailed down all the specific requirements, so you'll have to fill the gaps yourself and decide what other things you may want to implement (like is the account number going to be a mutable property, or "read-only" and so on).

See how many different ideas you can use from what we covered in the last section. 

My approach will end up creating two classes: a `TimeZone` class used to store the time zone name and offset definition (in hours and minutes), and a main class called `Account` that will have the following "public" interface:
- initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
- a first name property (read/write)
- a last name property (read/write)
- a full name property (computed, read-only)
- a balance property (read-only)
- an interest rate property (class level property)
- deposit, withdraw, pay_interest methods
- parse confirmation code

Class will have additional state and methods, but those will be used for implementation.

You should also remember to test your code! In the solutions I will introduce you to Python's `unittest` package. Even if you skip this project, at least review that video and/or notebook if you are unfamiliar with `unittest`.

In [1]:
from datetime import datetime
from pytz import timezone
from collections import namedtuple
import pytz
Confirmation = namedtuple('Confirmation', 'account_number, transaction_code, transaction_id, time_utc, time')

In [170]:
class Account:
    '''
    To create the account, pass in the following inputs:
    FirstName, LastName, Preferred Time Zone, starting balance
    '''
    default_time_zone = 'utc'
    interest_rate = 0.5 #percentage (%)
    transaction_number = 1
    _transaction_code = {
        'Interest':'I',
        'Deposit':'D',
        'Withdraw':'W',
        'Decline':'I'}
  
    def __init__ (self, first_name, last_name, account_number, time_zone='UTC', starting_balance=0 ):
        self._first_name = first_name
        self._last_name = last_name
        self._time_zone = time_zone
        self._balance = starting_balance
        self._account_number = account_number
        
        #Check_inputs
        if self.balance < 0.0:
            raise ValueError ('Balance cannot be less than zero')
        if time_zone not in pytz.all_timezones:
            raise ValueError ('TimeZone not found')
        self.validate_name (self._first_name, 'first_name')
        self.validate_name (self._last_name, 'last_name')
                       
    def deposit_interest (self):
        if self._balance > 0:
            self._balance = self._balance * ( 1 + Account.interest_rate/100)
            confirmation = 'I' + self.generate_confirmation ()
            Account.transaction_number = Account.transaction_number + 1
            return confirmation
        
        
    def deposit (self, deposit_amount):
        self._balance = self._balance + deposit_amount
        confirmation = Account._transaction_code['Deposit'] + self.generate_confirmation ()
        Account.transaction_number = Account.transaction_number + 1
        return confirmation
        
    def withdraw (self, withdraw_amount):
        if withdraw_amount < 0:
            raise ValueError ('Withdraw amount cannot be negative!!')
            return
        if self._balance - withdraw_amount < 0:
            Account.transaction_number = Account.transaction_number + 1
            confirmation = Account._transaction_code['Decline'] + self.generate_confirmation ()
            return confirmation
        else:
            self._balance = self._balance - withdraw_amount
            Account.transaction_number = Account.transaction_number + 1
            confirmation = Account._transaction_code['Withdraw'] + self.generate_confirmation ()
            return confirmation
    
    def convert_timezone (self, date, target_label):
        target_timezone = timezone (target_label)
        new_date  = date.astimezone(target_timezone)
        return new_date

    def generate_confirmation (self):
        confirmation_date = timezone(Account.default_time_zone).localize \
                    (datetime.today()).strftime('%Y%m%d%H%M%S')
        confirmation = str (self._account_number) + '-' + confirmation_date + '-' + \
        str(Account.transaction_number)  
        return confirmation
    
    def transaction_analysis (self, confirmation, target_timezone='UTC'):
        items = confirmation.split('-')
        transaction_id = items [0][0]
        account_number = items[0][0:]
        transaction_code = items[2]
        year = int ( items[1][0:4] )
        month = int ( items[1][4:6] )
        day = int (items[1][6:8])
        hour = int (items[1][8:10])
        minute = int (items[1][10:12])
        second = int (items[1][12:14])
        
        date_utc = datetime (year, month, day, hour, minute, second)
        if target_timezone not in pytz.all_timezones:
            raise ValueError ('TimeZone not found')
        date_preferred = self.convert_timezone (date_utc, target_timezone)
        conf = Confirmation (account_number, transaction_code, 
                             transaction_id, date_utc, date_preferred )
        return conf
    
    def validate_name(self, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty.')
        
    @property
    def account_number (self):
        return self._account_number
    
    @property
    def first_name (self):
        return self._first_name
    
    @property
    def last_name (self):
        return self._last_name
    
    @property
    def full_name (self):
        return self._first_name + ' ' + self._last_name
    
    @property
    def balance (self):
        return self._balance
    
    @property
    def time_zone (self):
        return self._time_zone
    
    def validate_and_set_name(self, property_name, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty.')
        setattr(self, property_name, value)

In [171]:
import unittest

In [172]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner =unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [173]:
class TestAccount(unittest.TestCase):
    
    def setUp(self):
        print ('running setup...')
        self.x = 100
        
    def test_ok(self):
        self.x = 200
        self.assertEqual(200,self.x)
    
    def test_2(self):
        self.assertEqual(200,self.x)
    
    def tearDown(self):
        print ('running teardown')
     

In [174]:
run_tests(TestAccount)

test_2 (__main__.TestAccount) ... FAIL
test_ok (__main__.TestAccount) ... 

running setup...
running teardown
running setup...
running teardown


ok

FAIL: test_2 (__main__.TestAccount)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-173-2af237090ab8>", line 12, in test_2
    self.assertEqual(200,self.x)
AssertionError: 200 != 100

----------------------------------------------------------------------
Ran 2 tests in 0.003s

FAILED (failures=1)


In [181]:
class TestAccount (unittest.TestCase):
    
    def setUp(self):
        self.account_number = '123345'
        self.first_name = 'First'
        self.last_name = 'Last'
        self.tz = 'Iran'
        self.balance = 1000.0
    
    def create_account (self):
        return Account (self.first_name, self.last_name, self.account_number, 
                     self.tz, self.balance)
    
    def test_create_account (self):
        a = self.create_account()
        self.assertEqual (self.account_number, a.account_number)
        self.assertEqual (self.first_name, a.first_name)
        self.assertEqual (self.last_name, a.last_name)
        self.assertEqual (self.first_name + ' ' + self.last_name, a.full_name)
        self.assertEqual (self.tz, a.time_zone)
        self.assertEqual (self.balance, a.balance)
        
    def test_create_account_blank_first_name(self):
        self.first_name = ''
        with self.assertRaises (ValueError):
            a = self.create_account()
            
    def test_create_negative_initial_balance(self):
        self.balance = -100
        with self.assertRaises (ValueError):
            a = self.create_account()
    
    def test_account_withdraw_ok (self):
        a = self.create_account()
        conf_code = a.withdraw(200)
        self.assertEqual(conf_code[0],'W')
        
    def test_account_withdraw_toomcuh (self):
        a = self.create_account()
        conf_code = a.withdraw(1200)
        self.assertEqual(conf_code[0],'I')


In [182]:
run_tests (TestAccount)

test_account_withdraw_ok (__main__.TestAccount) ... ok
test_account_withdraw_toomcuh (__main__.TestAccount) ... ok
test_create_account (__main__.TestAccount) ... ok
test_create_account_blank_first_name (__main__.TestAccount) ... ok
test_create_negative_initial_balance (__main__.TestAccount) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.003s

OK
